#import module

In [1]:
!pip install xml_to_dict #https://github.com/xthehatterx/xml_to_dict
!pip install sentencepiece
!pip install konlpy
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 64.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 35.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 50.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 86.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.4 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

#own location

In [4]:
%cd /content/drive/MyDrive/ColabNotebooks/산학_테스트
xml_sample_loc = 'xml_sample_20230519.csv'

/content/drive/MyDrive/ColabNotebooks/산학_테스트


#get 중복없는 keyword in xml sample file

In [5]:
xml_sample = pd.read_csv(xml_sample_loc)
keyword = xml_sample['keyword']

In [8]:
#get 중복 없는 keyword
keyword_unique = []

for i in range(len(keyword)):
  keyword_unique+=keyword[i].split('|')

keyword_unique = list(set(keyword_unique))

print(len(keyword_unique) )
print(keyword_unique[:20])

4176
['피크닉가방', '휴진', '촬영', '드론역할', '택배지연', '갤럭시', '워터파크', '상큼함', '원', '퍼스널컬러테스트', '아동', '해수욕', '아르타조나', '신선한', '서식하다', '떡메모지', '인기곡', '상승', '계산서', '인재']


#xml to json

In [ ]:
#same with https://github.com/miridi-sanhak/model/blob/main/preprocessing.py

import json
import math
from io import BytesIO

import pandas as pd
import requests
import xml_to_dict
import json
from PIL import Image


def process_bbox(XML_BBOX, IM_SIZE, SHEET_SIZE, angle, center):
    RATIO = IM_SIZE[0] / SHEET_SIZE[0]
    x1, y1, x2, y2 = map(float, XML_BBOX)
    x1, y1, x2, y2 = (x1 * RATIO, y1 * RATIO, x2 * RATIO, y2 * RATIO)
    center = (center[0] * RATIO, center[1] * RATIO)

    if angle != 0:
        angle = 360 - angle
        angle = math.radians(angle)
        # Calculate the center point of the bbox
        center_x, center_y = center
        # Calculate the distance from the center to each corner of the bbox
        distance_x = (x1 - center_x)
        distance_y = (y1 - center_y)
        # Apply rotation to the distances
        new_distance_x = distance_x * math.cos(angle) - distance_y * math.sin(angle)
        new_distance_y = distance_x * math.sin(angle) + distance_y * math.cos(angle)
        # Calculate the new corners after rotation
        x1 = center_x + new_distance_x
        y1 = center_y + new_distance_y
        x2 = center_x - new_distance_x
        y2 = center_y - new_distance_y

    x1, y1, x2, y2 = map(int, (x1, y1, x2, y2))

    return x1, y1, x2, y2

def get_render_bbox(text):
    if text['RenderPos'] == None:
        return []
    render_pos = json.loads(text['RenderPos'])
    render_bbox = []

    left, top, right, bottom = map(float, text['Position'].values())

    for render in render_pos['c']:
        x, a, w, y = map(float, [render['x'], render['a'], render['w'], render['y']])
        left_ = left + x
        right_ = left_ + w
        bottom_ = top + y
        top_ = bottom_ - a

        render_bbox.append((left_, top_, right_, bottom_))

    return render_bbox

def get_bbox(render_bbox):
    min_x = min([x[0] for x in render_bbox])
    min_y = min([x[1] for x in render_bbox])
    max_x = max([x[2] for x in render_bbox])
    max_y = max([x[3] for x in render_bbox])

    return min_x, min_y, max_x, max_y

def process_xml_dict(xml_dict, thumbnail):
    processed_json = {}
    processed_json['form'] = []

    SHEET_SIZE = tuple(map(int, xml_dict['SHEET']['SHEETSIZE'].values()))
    IM_SIZE = thumbnail.size

    # Process XML to json
    for i, text in enumerate(xml_dict['SHEET']['TEXT']):
        left, top, right, bottom = map(float, text['Position'].values())
        center = ((left + right) / 2, (top + bottom) / 2)

        render_bbox = get_render_bbox(text)
        if len(render_bbox) == 0: continue

        XML_BBOX = get_bbox(render_bbox)

        t = text['Text']
        x1, y1, x2, y2 = process_bbox(XML_BBOX, IM_SIZE, SHEET_SIZE, int(text['@Rotate']), center)

        processed_json['form'].append({
            "text": t,
            "box": [x1, y1, x2, y2],
            "font_id": int(text['Font']['@FamilyIdx']),
            "font_size": float(text['Font']['@Size']),
            "style": {
                "bold": text['Font']['Style']['@Bold'] == 'true',
                "italic": text['Font']['Style']['@Italic'] == 'true',
                "strikeout": text['Font']['Style']['@Strikeout'] == 'true',
                "underline": text['Font']['Style']['@Underline'] == 'true'
            },
            "linespace": float(text['Font']['@LineSpace']),
            "opacity": float(text['@Opacity']),
            "rotate": float(text['@Rotate']),
            "id": i
        })

        processed_json['form'][-1]['words'] = []

        render_pos = json.loads(text['RenderPos'])

        for j, bbox in enumerate(render_bbox):
            x1_, y1_, x2_, y2_ = process_bbox(bbox, IM_SIZE, SHEET_SIZE, int(text['@Rotate']), center)
            color = render_pos['c'][j]['f']
            color = color[4:-1]
            color = list(map(int, color.split(",")))
            processed_json['form'][-1]['words'].append({
                "text": render_pos['c'][j]['t'],
                "box": [x1_, y1_, x2_, y2_],
                "font_size": float(render_pos['c'][j]['s']),
                "letter_spacing": float(render_pos['c'][j]['ds']),
                "font_id": int(render_pos['c'][j]['yd']),
                "color": color
            })

    return processed_json

def process_xml(sheet_url, thumbnail_url):
    sample_thumbnail = Image.open(BytesIO(requests.get(thumbnail_url).content))
    sample_xml = requests.get(sheet_url).content.decode("utf-8")
    sample_json = xml_to_dict.XMLtoDict().parse(sample_xml)

    processed_json = process_xml_dict(sample_json, sample_thumbnail)

    return processed_json

def make_sample_json():
    # Read sample CSV and download thumbnail, XML
    df = pd.read_csv(xml_sample_loc)

    for i in range(len(df)):

      sample_sheet = df.iloc[i]

      try:
        processed_json = process_xml(sample_sheet['sheet_url'], sample_sheet['thumbnail_url'])

        filename = f"data/processed_sample_{i}.json"
        with open(filename, "w") as file_:
          json.dump(processed_json, file_, indent=4)
      except:
        print(f"error occurred in df line {i}")

In [ ]:
make_sample_json() #약 30분 소요. xml파일을 json으로 변환

#get text line in xml file

In [ ]:
#json파일에서 text들을 가져와 text_sum에 저장
import json
import os

text_sum = []

folder_path = "data/"  # 데이터 폴더 경로

# 폴더 내의 파일 목록 가져오기
file_list = os.listdir(folder_path)

for file_name in file_list:
    if file_name.endswith(".json"):  # .json 파일인 경우에만 처리
        file_path = os.path.join(folder_path, file_name)  # 파일 경로 생성

    try:
      with open(file_path, "r") as file_:
          data = json.load(file_)

      for i in range(len(data['form'])):
          if(data['form'][i]['text']!=None):
            text_sum.append(data['form'][i]['text'])  # 요소 추가

    except:
      print(f"error in {file_path}")


print(len(text_sum))
print(text_sum[0])

#train custom tokenizer (based on ke-t5)

In [ ]:
train_tokenization_data = np.concatenate((text_sum,keyword_unique)) #text와 keyword 합쳐서 한번에 train

print(len(train_tokenization_data))

def get_training_corpus(): #개수가 많으므로 1000개씩 끊어서 return
    return (
        train_tokenization_data[i : i + 1000]
        for i in range(0, len(train_tokenization_data), 1000)
    )

training_corpus = get_training_corpus()

In [ ]:
old_tokenizer = AutoTokenizer.from_pretrained("KETI-AIR/ke-t5-base-ko") #pre-trained된거 불러오기
old_tokenizer.vocab_size #64100

In [ ]:
tokenizer = old_tokenizer.train_new_from_iterator(training_corpus, 15000) #뒤 숫자는 우리 데이터에서 단어 개수
tokenizer.vocab_size #14602
tokenizer.save_pretrained("tokenizer_finetuned_ket5_by_xml_data") #저장

#see tokenized result (추후 학습과 관련 x)

In [ ]:
#tokenized by finetuned-ke-t5
#text_sum 안의 text들을 토큰화하여 text_token_list에 저장.

text_token_list = np.array([],dtype=object)

for idx , word in enumerate(train_tokenization_data):
  input_word = word
  try:
    tokens = tokenizer.tokenize(input_word)
    text_token_list = np.concatenate((text_token_list,tokens))
  except:
    print(f"error occur in input_word {input_word}, idx = {idx}")

print(text_token_list[:100])

In [ ]:
#keyword token과 text token들을 합쳐서 중복제거.

token_list = np.concatenate((text_token_list,keyword_unique))

token_list = list(set(token_list))

print(len(token_list) ) #==14602

print(token_list[:20])

#define custom tokenizer class

In [ ]:
from transformers import T5Tokenizer, T5TokenizerFast, PreTrainedTokenizer, PreTrainedTokenizerBase

import re
import sentencepiece as spm

# The special tokens of T5Tokenizer is hard-coded with <extra_id_{}>
# Created another class UDOPTokenizer extending it to add special visual tokens like <loc_{}>, etc.

#class UdopTokenizer(T5Tokenizer):
class UdopTokenizer(AutoTokenizer):

    def __init__(
        self,
        vocab_file,
        eos_token="</s>",
        unk_token="<unk>",
        pad_token="<pad>",
        extra_ids=100,
        loc_extra_ids=501,
        other_extra_ids=200,
        additional_special_tokens=[],
        sp_model_kwargs=None,
        **kwargs
    ):
        # Add extra_ids to the special token list
        if extra_ids > 0 and not "<extra_id_0>" in additional_special_tokens:
            additional_special_tokens = ["<extra_id_{}>".format(i) for i in range(extra_ids)]
            additional_special_tokens.extend(["<extra_l_id_{}>".format(i) for i in range(extra_ids)])
            additional_special_tokens.extend(["</extra_l_id_{}>".format(i) for i in range(extra_ids)])
            additional_special_tokens.extend(["<extra_t_id_{}>".format(i) for i in range(extra_ids)])
            additional_special_tokens.extend(["</extra_t_id_{}>".format(i) for i in range(extra_ids)])

        if loc_extra_ids > 0 and not "<loc_0>" in additional_special_tokens:
            additional_special_tokens.extend(["<loc_{}>".format(i) for i in range(loc_extra_ids)])

        if other_extra_ids > 0 and not "<other_0>" in additional_special_tokens:
            additional_special_tokens.extend(["<other_{}>".format(i) for i in range(other_extra_ids)])
        print(additional_special_tokens)
        PreTrainedTokenizer.__init__(
            self,
            eos_token=eos_token,
            unk_token=unk_token,
            pad_token=pad_token,
            extra_ids=extra_ids,
            additional_special_tokens=additional_special_tokens,
            **kwargs,
        )

        self.sp_model_kwargs = {} if sp_model_kwargs is None else sp_model_kwargs

        self.vocab_file = vocab_file
        self._extra_ids = extra_ids
        self._loc_extra_ids = loc_extra_ids
        self._other_extra_ids = other_extra_ids

        self.sp_model = spm.SentencePieceProcessor(**self.sp_model_kwargs)
        self.sp_model.Load(vocab_file)

    @property
    def vocab_size(self):
        return self.sp_model.get_piece_size() + self._extra_ids * 5 + self._loc_extra_ids + self._other_extra_ids

    def get_vocab(self):
        vocab = {self.convert_ids_to_tokens(
            i): i for i in range(self.vocab_size)}
        vocab.update(self.added_tokens_encoder)
        return vocab

    def _convert_token_to_id(self, token):
        """ Converts a token (str) in an id using the vocab. """
        if token.startswith("<extra_id_"):
            match = re.match(r"<extra_id_(\d+)>", token)
            num = int(match.group(1))
            return self.vocab_size - num - 1 - self._other_extra_ids - self._loc_extra_ids - self._extra_ids * 4
        elif token.startswith("<extra_l_id_"):
            match = re.match(r"<extra_l_id_(\d+)>", token)
            num = int(match.group(1))
            return self.vocab_size - num - 1 - self._other_extra_ids - self._loc_extra_ids - self._extra_ids * 3
        elif token.startswith("</extra_l_id_"):
            match = re.match(r"</extra_l_id_(\d+)>", token)
            num = int(match.group(1))
            return self.vocab_size - num - 1 - self._other_extra_ids - self._loc_extra_ids - self._extra_ids * 2
        elif token.startswith("<extra_t_id_"):
            match = re.match(r"<extra_t_id_(\d+)>", token)
            num = int(match.group(1))
            return self.vocab_size - num - 1 - self._other_extra_ids - self._loc_extra_ids - self._extra_ids
        elif token.startswith("</extra_t_id_"):
            match = re.match(r"</extra_t_id_(\d+)>", token)
            num = int(match.group(1))
            return self.vocab_size - num - 1 - self._other_extra_ids - self._loc_extra_ids
        elif token.startswith("<loc_"):
            match = re.match(r"<loc_(\d+)>", token)
            num = int(match.group(1))
            return self.vocab_size - num - 1 - self._other_extra_ids
        elif token.startswith("<other_"):
            match = re.match(r"<other_(\d+)>", token)
            num = int(match.group(1))
            return self.vocab_size - num - 1

        return self.sp_model.piece_to_id(token)

    def _convert_id_to_token(self, index):
        """Converts an index (integer) in a token (str) using the vocab."""
        if index < self.sp_model.get_piece_size():
            token = self.sp_model.IdToPiece(index)
        else:

            if index > self.sp_model.get_piece_size() + self._extra_ids * 5 + self._loc_extra_ids - 1:
                index_loc = self.vocab_size - 1 - index
                token = f"<other_{index_loc}>"
            elif index > self.sp_model.get_piece_size() + self._extra_ids * 5 - 1:
                index_loc = self.vocab_size - self._other_extra_ids - 1 - index
                token = f"<loc_{index_loc}>"
            elif index > self.sp_model.get_piece_size() + self._extra_ids * 4 - 1:
                token = "</extra_t_id_{}>".format(self.vocab_size - self._other_extra_ids - self._loc_extra_ids - 1 - index)
            elif index > self.sp_model.get_piece_size() + self._extra_ids * 3 - 1:
                token = "<extra_t_id_{}>".format(self.vocab_size - self._other_extra_ids - self._loc_extra_ids - self._extra_ids - 1 - index)
            elif index > self.sp_model.get_piece_size() + self._extra_ids * 2 - 1:
                token = "</extra_l_id_{}>".format(self.vocab_size - self._other_extra_ids - self._loc_extra_ids - self._extra_ids * 2 - 1 - index)
            elif index > self.sp_model.get_piece_size() + self._extra_ids - 1:
                token = "<extra_l_id_{}>".format(self.vocab_size - self._other_extra_ids - self._loc_extra_ids - self._extra_ids * 3 - 1 - index)
            elif index > self.sp_model.get_piece_size() - 1:
                token = "<extra_id_{}>".format(self.vocab_size - self._other_extra_ids - self._loc_extra_ids - self._extra_ids * 4 - 1 - index)
            else:
                raise
        return token

#test tokenizer class

In [ ]:
my_tokenizer = UdopTokenizer.from_pretrained("tokenizer_finetuned_ket5_by_xml_data")

In [ ]:
my_tokenizer.vocab_size
my_tokenizer.decode([4321,1231,1050])